In [12]:
import pandas as pd
import sqlite3
import os
import numpy as np

In [19]:
existing_excel_path=r'D:\BC-World\CryptoTrans\BS-Trade\Master\Master_BinanceTradeHistory.xlsx'
data_base_file=f'DB_BN_MyTrade.db'

In [20]:
df_existing=pd.read_excel(existing_excel_path,sheet_name='Master_BuySell')

In [21]:

cols_name={'DateTime(UTC)':'datetimeUTC','Symbol':'symbol','Type':'type','Price':'price','Amount':'qty','Total':'total'}
df_existing=df_existing.rename(columns=cols_name)
df_existing['pair']='BUSD'
df_existing['symbol_pair']=df_existing.apply( lambda x: f"{x.symbol}{x.pair}",axis=1)
df_existing['commission']=0
df_existing['commissionAsset']='BNB'
df_existing['datetimeUTC']=df_existing['datetimeUTC'].dt.strftime("%Y-%m-%d %H:%M:%S")
cols=['datetimeUTC','symbol_pair','symbol','pair','type','price','qty','total','commission','commissionAsset']
df_existing=df_existing[cols]
df_existing.head()

,datetimeUTC,symbol_pair,symbol,pair,type,price,qty,total,commission,commissionAsset
0,2022-08-13 01:18:00,MATICBUSD,MATIC,BUSD,SELL,0.9400,22.0000,20.68000,0,BNB
1,2022-08-13 00:59:29,BTCBUSD,BTC,BUSD,SELL,24500.0000,0.0005,12.25000,0,BNB
2,2022-08-12 17:08:40,MATICBUSD,MATIC,BUSD,BUY,0.9177,22.0000,20.18940,0,BNB
3,2022-08-12 16:49:52,BNBBUSD,BNB,BUSD,BUY,322.6000,0.0460,14.83960,0,BNB
4,2022-08-12 16:40:13,ETHBUSD,ETH,BUSD,BUY,1898.9000,0.0076,14.43164,0,BNB


In [22]:
df_existing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 807 entries, 0 to 806
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   datetimeUTC      807 non-null    object 
 1   symbol_pair      807 non-null    object 
 2   symbol           807 non-null    object 
 3   pair             807 non-null    object 
 4   type             807 non-null    object 
 5   price            807 non-null    float64
 6   qty              807 non-null    float64
 7   total            807 non-null    float64
 8   commission       807 non-null    int64  
 9   commissionAsset  807 non-null    object 
dtypes: float64(3), int64(1), object(6)
memory usage: 63.2+ KB


In [23]:
#df_existing.to_csv('data/binance_hist_trade.csv',index=False)
print("Connect SQLite")
conn = sqlite3.connect(os.path.abspath(data_base_file))
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

Connect SQLite


In [24]:
def insertTradeTransaction(recordList):
    try:
        sqliteConnection = sqlite3.connect(os.path.abspath(data_base_file))
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        sqlite_insert_query = """
        INSERT INTO hist_trade
         VALUES (?,?,?,?,?,?,?,?,?,?);
         """

        cursor.executemany(sqlite_insert_query, recordList)
        sqliteConnection.commit()
        print("Total", cursor.rowcount, "Add transaction successfully")
        sqliteConnection.commit()
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert multiple records to table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")




In [25]:
recordsToInsert=list(df_existing.to_records(index=False))
insertTradeTransaction(recordsToInsert)

Connected to SQLite
Total 807 Add transaction successfully
The SQLite connection is closed
